In [ ]:
!pip -q install google-generativeai==0.3.0
!pip -q install google-ai-generativelanguage==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.9/145.9 kB 2.7 MB/s eta 0:00:00


In [ ]:
import numpy as np # linear algebra
# !pip install datasets --quiet
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import seaborn as sns

In [ ]:
!wget -O train.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv"

!wget -O trial.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_trial.tsv"

In [ ]:
train, trial = df = pd.read_table('train.tsv'), pd.read_table('trial.tsv')

train = train[train["corpus"] == "bible"].drop(["corpus", "id"], axis = 1)
trial = trial[trial["subcorpus"] == "bible"].drop(["subcorpus", "id"], axis = 1)

In [ ]:
from google.colab import userdata

In [ ]:
# setup
import google.generativeai as genai
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown

GOOGLE_AI_STUDIO = userdata.get('key')

genai.configure(api_key=GOOGLE_AI_STUDIO)

In [ ]:
for m in genai.list_models():
    print(m.name)
    print(m.supported_generation_methods)

models/chat-bison-001
['generateMessage', 'countMessageTokens']
models/text-bison-001
['generateText', 'countTextTokens', 'createTunedTextModel']
models/embedding-gecko-001
['embedText', 'countTextTokens']
models/gemini-1.0-pro
['generateContent', 'countTokens']
models/gemini-1.0-pro-001
['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.0-pro-latest
['generateContent', 'countTokens']
models/gemini-1.0-pro-vision-latest
['generateContent', 'countTokens']
models/gemini-1.5-pro-latest
['generateContent', 'countTokens']
models/gemini-pro
['generateContent', 'countTokens']
models/gemini-pro-vision
['generateContent', 'countTokens']
models/embedding-001
['embedContent']
models/text-embedding-004
['embedContent']
models/aqa
['generateAnswer']


In [ ]:
!pip install datasets --quiet
from datasets import Dataset

data = Dataset.from_pandas(trial[:35])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00


In [ ]:
# Set up the model
generation_config = {
  "temperature": 0.1,
  "top_p": 0.6,
  "top_k": 1,
  "max_output_tokens": 100,
}


safe = [
        {
            "category": "HARM_CATEGORY_DANGEROUS",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE",
        },
    ]

from tqdm import tqdm

model = genai.GenerativeModel(model_name="gemini-1.0-pro-latest",
                              generation_config=generation_config, safety_settings = safe)

result = []

for row in tqdm(data):

  prompt_parts = [
    "Evaluate the lexical complexity of the token by considering its interaction and relationship " \
    "with the other elements within the sentence. Reflect on how this token contributes to the overall " \
    "sentence meaning and structure. Provide a complexity score from 0 to 1, reflecting the lowest to " \
    "highest complexity, guided by the mean score of 0.301 and a standard deviation of 0.133.\n" \
    "Output only one single score number"
    f"\nSentence: {row['sentence']}" \
    f"\nToken: {row['token']}"
]

  response = model.generate_content(prompt_parts)
  # print(response.text)
  result.append(float(response.text))

100%|██████████| 35/35 [01:09<00:00,  1.98s/it]


In [ ]:
from scipy.stats import pearsonr

print(pearsonr(result, trial[:35]["complexity"]))

PearsonRResult(statistic=0.41062403253124125, pvalue=0.014277201494427804)


In [ ]:
from scipy.stats import pearsonr

# on seed = 9, temperature = 0.1, top_p = 0.5, top_k = 1
print(pearsonr(result, trial[:35]["complexity"]))

PearsonRResult(statistic=0.4109067400673655, pvalue=0.0142038747685761)
